In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.torch as ptorch
import torch.optim as opts
import numpy as np
import plasma.torch.training as training

from torch.utils.data import DataLoader

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
@training.wrappers.NanChecker()
@training.wrappers.Tensorboard('test')
@training.wrappers.GradientClipping()
class TestTrainer(ptorch.training.Trainer):

    def __init__(self):
        super().__init__()

        self.max_epoch = 2

        model = ptorch.nn.Linear(1, 1)
        self.model = model
        self.optimizer = opts.SGD(model.parameters())
        self.scheduler = None
    
    def init_train_loader(self) -> DataLoader:
        ds = ptorch.data.AdhocData([1.0]*10, lambda x: x)
        return ds.get_torch_loader(batch_size=1, workers=0)
    
    def forward(self, i, inputs):
        self.model.zero_grad()
        y = self.model(inputs[np.newaxis].float())

        return (y - inputs).mean()
    
    def backward(self, i, inputs, objective_val):
        objective_val.backward()
        self.optimizer.step()
    
    def optimize(self, i, inputs, objective_val):
        super().optimize(i, inputs, objective_val)

In [4]:
trainer = TestTrainer()

In [5]:
trainer.run()

epoch: 100%|██████████| 2/2 [00:00<00:00, 91.71it/s]
